# Use of Random Forest to select the most important channels
* This notebook will use random forest the rank the channels from the most important to least importants. 
* The goal is to see which channels are the most important which are the least important and drop the least important channels and classfy again.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


In [ ]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [ ]:
epoch=remove_artifacts_and_create_epochs('D:\\USB Drive\\NewEEG-200s\\SA20140206\\20140206_1151.set')

In [ ]:
def removebadchannel(epoch):
    epoch.drop_channels(['Lm','Rm','VEOG'])
    epoch.drop_channels(['STI 014'])

In [ ]:
 removebadchannel(epoch)

In [ ]:
newdata=epoch.get_data()
epoch.get_data().shape

In [ ]:
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [ ]:
def findchanel (newdata):
    arrays=[]
    for epochs in newdata:
        dataof50=[]
        sum1=[]
        sum2=[]
        index=[]
        k=0
        for channel in epochs:
            dataof50.append(channel)
        arrayofdata50=np.array(dataof50)
        for channel in arrayofdata50:
            sum1.append(np.sum(channel))
        arraysum1=np.array(sum1)
        mean1=np.sum(arraysum1)/125
        std=np.std(arraysum1)
        dis1=mean1-std
        dis2=mean1+std
        j=0
        channel1=[]
        for k in range(len(arraysum1)):
            if (arraysum1[k]<dis1 or arraysum1[k]>dis2) and j<30:
                j=j+1
                sum2.append(arraysum1[k])
                index.append(k)
                x=k
                channel1.append(arrayofdata50[k])
        while (j<30):
            channel1.append(arrayofdata50[x])
            j=j+1
        arrays.append(channel1)
        arrayofdata=np.array(arrays)
        print(arrayofdata.shape)
    return arrayofdata

In [ ]:
arrayofdata=findchanel(newdata)

In [ ]:
epochs=filter_event_id(epoch)
print(epoch.events[:,-1])

In [ ]:
def classify_test_KNN(X,y):

    clf = make_pipeline(Vectorizer(),
                        StandardScaler(),
                        RandomForestRegressor(n_estimators = 500, random_state = 42))
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
    clf.fit(X_train, y_train)
    
    score = clf.score(X_test, y_test)
    
    return score


In [ ]:
X = arrayofdata
y = epoch.events[:,-1]

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators = 10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
print(y_pred.round(0), y_test)

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    print(i)
    channel.append(np.mean(importances[i:i+500]))
    channel1=np.array(channel)

In [ ]:
channel=np.array(channel)
channel.shape

In [ ]:
import pandas as pd
importances = list(channel)
feature_imp = pd.Series(channel,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
def averagechannel (epoch):
    X = epoch.get_data()
    y = epoch.events[:,-1]
    X.shape

    data1=[]
    for event in X:
        data=[]
        for channel in event:

            data.append(np.mean(channel))
        data1.append(data)
    data=np.array(data1)
    print(data.shape)
    return data

In [ ]:
X = data
y = epoch.events[:,-1]
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators = 5, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
print(importances.shape)
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =500, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
print(importances.shape)
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
epoch.drop_channels(['R8','LL10','LL13','RR3','Z11','L5','Z12','RC2'])

In [ ]:
data=averagechannel(epoch)
X = data
y = epoch.events[:,-1]
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =5, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
print(importances.shape)
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(X, round(importance, 2)) for X, importance in zip(X, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
print(score)

In [ ]:
clf = make_pipeline(Vectorizer(),
                    StandardScaler(),
                    KNN(n_neighbors=13))
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)

In [ ]:
y = epoch.events[:,-1]
print(y)

In [ ]:
epoch.drop_channels(['RC6','Z4', 'Z13'])

In [ ]:
data=averagechannel(epoch)
X = data
y = epoch.events[:,-1]
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =50, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
print(importances.shape)
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=epoch.ch_names).sort_values(ascending=False)
feature_imp